In [1]:
import random
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from rdkit import DataStructs
from rdkit.Chem import PandasTools, AllChem
import tensorflow as tf

In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    tf.random.set_seed(seed)

seed_everything(42) # Seed 고정

In [3]:
train = pd.read_csv('./train.csv')
train = train.groupby('SMILES').mean().reset_index()
test = pd.read_csv('./test.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_17632\946614270.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  train = train.groupby('SMILES').mean().reset_index()


In [3]:
import seaborn as sns
for i in train.columns[3:]:
    sns.distplot(train[i])
    sns.distplot(test[i])
    plt.show()

NameError: name 'train' is not defined

In [4]:
train = train[train.Num_H_Donors < 7]
train = train[train.Num_RotatableBonds < 20]
train = train[train.AlogP > -3]
print(train.describe())
print(test.describe())

# test.describe()

               MLM          HLM        AlogP  Molecular_Weight  \
count  3456.000000  3456.000000  3456.000000       3456.000000   
mean     37.337354    53.139955     2.840858        362.394647   
std      35.651062    36.057320     1.576301         75.452551   
min       0.000000     0.000000    -2.414000        172.180000   
25%       2.453000    16.044750     1.833000        304.346000   
50%      26.145500    58.083000     2.879000        349.426500   
75%      70.722500    87.772500     3.954750        408.493250   
max     131.720000   135.336000     9.190000        730.737000   

       Num_H_Acceptors  Num_H_Donors  Num_RotatableBonds         LogD  \
count      3456.000000   3456.000000         3456.000000  3456.000000   
mean          3.984086      1.083912            4.527778     2.680341   
std           1.441992      0.924293            2.231814     1.641791   
min           0.000000      0.000000            0.000000    -3.322000   
25%           3.000000      0.000000    

In [5]:
PandasTools.AddMoleculeColumnToFrame(train,'SMILES','Molecule')
PandasTools.AddMoleculeColumnToFrame(test,'SMILES','Molecule')
def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 8, nBits=4096*2)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

train["FPs"] = train.Molecule.apply(mol2fp)
test["FPs"] = test.Molecule.apply(mol2fp)

droping = []
tp = pd.DataFrame(train['SMILES'].str.contains('e'))
droping.extend(list(tp[tp['SMILES'] == True].index))
tp = pd.DataFrame(train['SMILES'].str.contains('I'))
droping.extend(list(tp[tp['SMILES'] == True].index))
tp = pd.DataFrame(train['SMILES'].str.contains('6'))
droping.extend(list(tp[tp['SMILES'] == True].index))

train = train.drop(droping)

feed = [list(i) for i in train.FPs]
# feed.extend([list(reversed(i)) for i in train.FPs])
feed = np.array(feed)
feed_test = np.array([list(i) for i in test.FPs])

In [7]:
feed.shape

(3441, 8192)

In [84]:
tp = [[j for j in i] for i in train['SMILES']]
ans = []
for i in tp:
    ans.extend(i)
    
import collections
data = pd.DataFrame([collections.Counter(ans).keys(),collections.Counter(ans).values()]).transpose()
data.columns = ['atom','counter']
data.sort_values('counter',ascending=False).reset_index(drop=True)

,atom,counter
0,c,39264
1,C,25700
2,(,14618
3,),14618
4,1,10186
5,O,8400
6,2,8150
7,n,6743
8,=,5689
9,N,4742


In [85]:
test['SMILES'] = [i.replace('P','') for i in test['SMILES']]
tp = [[j for j in i] for i in test['SMILES']]
ans = []
for i in tp:
    ans.extend(i)
    
import collections
data = pd.DataFrame([collections.Counter(ans).keys(),collections.Counter(ans).values()]).transpose()
data.columns = ['atom','counter']
data.sort_values('counter',ascending=False).reset_index(drop=True)

,atom,counter
0,c,5169
1,C,3720
2,),1926
3,(,1926
4,1,1382
5,2,1164
6,O,1143
7,n,965
8,=,751
9,N,683


In [86]:
tp = [[j for j in i] for i in test['SMILES']]
ans = []
for i in tp:
    ans.extend(i)
    
import collections
data_t = pd.DataFrame([collections.Counter(ans).keys(),collections.Counter(ans).values()]).transpose()
data_t.columns = ['atom','counter']
data_t.sort_values('counter',ascending=False).reset_index(drop=True)

vectorizer_rule = pd.merge(data,data_t,how='left',on='atom').sort_values('counter_x',ascending=False).reset_index(drop=True)
vectorizer_rule

,atom,counter_x,counter_y
0,c,5169,5169
1,C,3720,3720
2,),1926,1926
3,(,1926,1926
4,1,1382,1382
5,2,1164,1164
6,O,1143,1143
7,n,965,965
8,=,751,751
9,N,683,683


In [300]:
tp = [[j for j in i] for i in train['SMILES']]

In [301]:
vectorizer_rule = {vectorizer_rule.loc[i,'atom']:i for i in range(len(vectorizer_rule))}

AttributeError: 'dict' object has no attribute 'loc'

In [302]:
max([len([vectorizer_rule[j] for j in i]) for i in tp])

99

In [303]:
vector = [[vectorizer_rule[j] for j in i] for i in tp]
# max_len = max([len([vectorizer_rule[j] for j in i]) for i in tp])
# for i in vector:
#     i.extend([0 for _ in range(max_len-len(i))])
    
train['SMILES'] = vector
train['lenS'] = [len(i) for i in vector]
train.describe()

,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,lenS
count,3441.000000,3441.000000,3441.000000,3441.000000,3441.000000,3441.000000,3441.000000,3441.000000,3441.000000,3441.000000
mean,37.251523,53.116251,2.839206,362.134844,3.983726,1.083987,4.530950,2.678582,79.280372,44.199070
std,35.646792,36.056933,1.578057,75.323242,1.441604,0.923959,2.232487,1.643904,27.899518,10.238957
min,0.000000,0.000000,-2.414000,172.180000,0.000000,0.000000,0.000000,-3.322000,3.240000,18.000000
25%,2.442000,16.029000,1.829000,304.343000,3.000000,0.000000,3.000000,1.609000,59.810000,37.000000
50%,25.940000,58.066000,2.878000,349.339000,4.000000,1.000000,4.000000,2.703000,77.230000,43.000000
75%,70.700000,87.770000,3.954000,408.371000,5.000000,2.000000,6.000000,3.857000,96.240000,50.000000
max,131.720000,135.336000,9.190000,730.737000,12.000000,6.000000,17.000000,9.190000,211.950000,99.000000


In [304]:
vector = [[vectorizer_rule[j] for j in i] for i in tp]
max_len = max([len([vectorizer_rule[j] for j in i]) for i in tp])
for i in vector:
    i.extend([0 for _ in range(max_len-len(i))])
    
from sklearn.decomposition import PCA
pca = PCA(n_components=64)
train_vec = pca.fit_transform(vector)
train_vec

array([[-7.78030031e+00,  9.98210758e+00,  8.76794067e+00, ...,
        -1.04019346e-01,  4.65155052e-02,  2.17907754e-01],
       [-2.26054469e+00,  8.95656823e-01,  7.20882070e+00, ...,
         1.21835386e-01, -3.25413308e-02,  8.87940633e-02],
       [-7.85477586e-01,  2.72635322e+00,  6.86334641e+00, ...,
        -5.06530906e-01, -5.28209011e-02, -1.04249029e-01],
       ...,
       [ 1.45719132e+01, -6.25587741e+00, -5.60715474e+00, ...,
        -1.36688776e-01,  2.87135099e-01, -6.22903404e-02],
       [-4.21368896e+00, -6.23270295e+00, -4.30323426e+00, ...,
         2.05413789e-01,  7.09767238e-02,  1.68442555e-01],
       [ 7.85832981e+00,  6.38205219e-01, -2.15823686e+00, ...,
         1.41589275e-02,  9.06890244e-02,  1.06039933e-01]])

In [325]:
np.concatenate((train_vec,feed),axis=1).shape

(3441, 8256)

In [8]:
# This function keeps the initial learning rate for the first ten epochs
# and decreases it exponentially after that.
def scheduler(epoch, lr):
    if epoch%5 == 0:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])
# model.compile(tf.keras.optimizers.SGD(), loss='mse')
# round(model.optimizer.lr.numpy(), 5)
# callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
# history = model.fit(np.arange(100).reshape(5, 20), np.zeros(5),
#                     epochs=15, callbacks=[callback], verbose=0)
# round(model.optimizer.lr.numpy(), 5)


In [12]:
feed = pd.read_csv('./data.csv')
feed = feed.drop(['Unnamed: 0','id','SMILES','HLM'],axis=1)
feed

,MLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,s_form,r_form,...,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216
0,26.010,3.259,400.495,5,2,8,3.259,117.37,0,0,...,2,0,1,0,0,0,4,0,0,0
1,29.270,2.169,301.407,2,1,2,2.172,73.47,0,0,...,2,0,0,0,0,0,4,0,0,0
2,5.710,4.771,494.652,6,0,5,3.475,92.60,0,0,...,4,0,0,0,0,0,6,0,0,0
3,27.640,1.335,419.422,4,1,7,1.335,78.95,0,0,...,0,1,0,0,1,0,2,0,0,0
4,82.107,1.954,547.707,7,4,9,0.464,144.42,0,0,...,0,0,0,0,0,0,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2726,1.556,3.409,396.195,3,1,5,3.409,64.74,0,0,...,0,2,0,0,0,0,0,0,0,0
2727,35.560,1.912,359.381,4,1,3,1.844,77.37,0,0,...,0,0,0,0,0,0,11,0,0,0
2728,56.150,1.941,261.320,3,1,6,2.124,70.14,0,0,...,1,0,0,0,0,0,3,0,0,0
2729,0.030,0.989,284.696,5,1,5,0.989,91.51,0,0,...,0,0,0,0,0,0,2,0,0,0


In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, GlobalAvgPool1D, Flatten, Dropout, Normalization, AveragePooling1D, AveragePooling2D, GaussianNoise
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.optimizers import Adam,Adagrad


X_numeric = feed.drop('MLM',axis=1)
y_regression = np.array(list(feed['MLM']))

X_train, X_test, y_train, y_test = train_test_split(X_numeric, y_regression, test_size=0.1, random_state=219842194)

model = Sequential()
# model.add(Dense(units=256, activation='relu', input_dim=X_numeric.shape[1]))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu', input_shape=(X_numeric.shape[1], 1)))
model.add(AveragePooling1D(pool_size=4))
model.add(Conv1D(filters=128, kernel_size=4, activation='relu', input_shape=(X_numeric.shape[1], 1)))
model.add(AveragePooling1D(pool_size=4))
model.add(Dropout(0.5))

# model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
# model.add(AveragePooling1D(pool_size=8))
# model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))

custom_learning_rate = 0.005
adam_optimizer = Adam(learning_rate=custom_learning_rate)
model.compile(optimizer=adam_optimizer, loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

model.summary()
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit(X_train, y_train, epochs=20, batch_size=2, validation_split=0.1)

# 모델 평가 (RMSE는 평가 함수에 RMSE 지표가 포함되어 있어야 함)
loss, rmse = model.evaluate(X_test, y_test)
print(f"평가 손실 (MSE): {loss}")
print(f"평가 RMSE: {rmse}")



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 230, 64)           320       
                                                                 
 average_pooling1d_4 (Averag  (None, 57, 64)           0         
 ePooling1D)                                                     
                                                                 
 conv1d_5 (Conv1D)           (None, 54, 128)           32896     
                                                                 
 average_pooling1d_5 (Averag  (None, 13, 128)          0         
 ePooling1D)                                                     
                                                                 
 dropout_4 (Dropout)         (None, 13, 128)           0         
                                                                 
 flatten_2 (Flatten)         (None, 1664)             

In [ ]:
# history.history

In [15]:
loss, rmse = model.evaluate(X_test, y_test)
print(f"평가 손실 (MSE): {loss}")
print(f"평가 RMSE: {rmse}")


9/9 [==============================] - 0s 2ms/step - loss: 632.9708 - root_mean_squared_error: 25.1589
평가 손실 (MSE): 632.9708251953125
평가 RMSE: 25.158910751342773


In [25]:
# tp = [[j for j in i] for i in test['SMILES']]
# vector = [[vectorizer_rule[j] for j in i] for i in tp]
# max_len = max([len([vectorizer_rule[j] for j in i]) for i in tp])
# for i in vector:
#     i.extend([0 for _ in range(max_len-len(i))])
    
# from sklearn.decomposition import PCA
# pca = PCA(n_components=64)
# test_vec = pca.fit_transform(vector)
# test_vec
feed_test = pd.read_csv('./data_t.csv').drop(['Unnamed: 0'],axis=1)
feed_test
mlm = model.predict(feed_test)

16/16 [==============================] - 0s 2ms/step


In [34]:
feed = pd.read_csv('./hlm_train.csv').drop('Unnamed: 0',axis=1)
feed = feed.drop(['id','SMILES','MLM'],axis=1)

In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, GlobalMaxPooling1D
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.optimizers import Adam


# 가상의 정형 데이터 생성
X_numeric = feed.drop('HLM',axis=1)  # 100개의 샘플과 5개의 특성
y_regression = feed['HLM'] # 회귀 타겟 값

X_train, X_test, y_train, y_test = train_test_split(X_numeric, y_regression, test_size=0.1, random_state=219842194)

model = Sequential()
# model.add(Dense(units=256, activation='relu', input_dim=X_numeric.shape[1]))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu', input_shape=(X_numeric.shape[1], 1)))
model.add(AveragePooling1D(pool_size=4))
model.add(Conv1D(filters=128, kernel_size=4, activation='relu', input_shape=(X_numeric.shape[1], 1)))
model.add(AveragePooling1D(pool_size=4))
model.add(Dropout(0.5))

# model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
# model.add(AveragePooling1D(pool_size=8))
# model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))

custom_learning_rate = 0.005
adam_optimizer = Adam(learning_rate=custom_learning_rate)
model.compile(optimizer=adam_optimizer, loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

model.summary()
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit(X_train, y_train, epochs=20, batch_size=2, validation_split=0.1)

# 모델 평가 (RMSE는 평가 함수에 RMSE 지표가 포함되어 있어야 함)
loss, rmse = model.evaluate(X_test, y_test)
print(f"평가 손실 (MSE): {loss}")
print(f"평가 RMSE: {rmse}")


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_10 (Conv1D)          (None, 230, 64)           320       
                                                                 
 average_pooling1d_6 (Averag  (None, 57, 64)           0         
 ePooling1D)                                                     
                                                                 
 conv1d_11 (Conv1D)          (None, 54, 128)           32896     
                                                                 
 average_pooling1d_7 (Averag  (None, 13, 128)          0         
 ePooling1D)                                                     
                                                                 
 dropout_10 (Dropout)        (None, 13, 128)           0         
                                                                 
 flatten_5 (Flatten)         (None, 1664)             

C:\Users\User\anaconda3\envs\git_repo\lib\site-packages\keras\engine\data_adapter.py:1508: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


1064/1064 [==============================] - 22s 20ms/step - loss: 1785.4713 - root_mean_squared_error: 42.2548 - val_loss: 1092.6389 - val_root_mean_squared_error: 33.0551
Epoch 2/20
1064/1064 [==============================] - 22s 20ms/step - loss: 1437.2028 - root_mean_squared_error: 37.9105 - val_loss: 1389.3860 - val_root_mean_squared_error: 37.2745
Epoch 3/20
1064/1064 [==============================] - 22s 21ms/step - loss: 1420.6001 - root_mean_squared_error: 37.6908 - val_loss: 974.9933 - val_root_mean_squared_error: 31.2249
Epoch 4/20
1064/1064 [==============================] - 26s 24ms/step - loss: 1415.8217 - root_mean_squared_error: 37.6274 - val_loss: 1085.1008 - val_root_mean_squared_error: 32.9409
Epoch 5/20
1064/1064 [==============================] - 24s 22ms/step - loss: 1356.0553 - root_mean_squared_error: 36.8247 - val_loss: 1019.2148 - val_root_mean_squared_error: 31.9251
Epoch 6/20
1064/1064 [==============================] - 28s 27ms/step - loss: 1255.7723 - ro

In [37]:
loss, rmse = model.evaluate(X_test, y_test)
print(f"평가 손실 (MSE): {loss}")
print(f"평가 RMSE: {rmse}")

9/9 [==============================] - 0s 2ms/step - loss: 738.9925 - root_mean_squared_error: 27.1844
평가 손실 (MSE): 738.9924926757812
평가 RMSE: 27.184415817260742


In [38]:
# tp = [[j for j in i] for i in test['SMILES']]
# vector = [[vectorizer_rule[j] for j in i] for i in tp]
# max_len = max([len([vectorizer_rule[j] for j in i]) for i in tp])
# for i in vector:
#     i.extend([0 for _ in range(max_len-len(i))])
    
# from sklearn.decomposition import PCA
# pca = PCA(n_components=64)
# test_vec = pca.fit_transform(vector)
# test_vec

feed_test = pd.read_csv('./data_t.csv').drop(['Unnamed: 0'],axis=1)
feed_test
hlm = model.predict(feed_test)

16/16 [==============================] - 0s 2ms/step


In [39]:
sub = pd.read_csv('./sample_submission.csv')
sub['MLM'] = mlm
sub['HLM'] = hlm
sub = sub.set_index('id')
sub.to_csv('./submission.csv')